Final Table Example


| restaurant | category | name    | calories | carbs | fat |
|------------|----------|---------|----------|-------|-----|
| McDonald's | Burgers  | Big Mac | 540      | 45    | 29  |
| Burger King | Burgers  | Whopper | 900      | 51    | 57  |
| ... | ...  | ... | ...      | ...    | ...  |
| Chili's | Ribs  | Shiner Bock® BBQ Ribs | 2310      | 168    | 123  |

In [11]:
# Import libaries
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Step 1: Create a soup object from the home page

In [38]:
# create url
url = 'http://chubbygrub.com/'

# read content
res = requests.get(url)

# check status code
res.status_code

200

In [13]:
# create soup object
soup = BeautifulSoup(res.content, 'lxml')

### Step 2: Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [87]:
# create list for restaurant + slugs
restaurants = []
for row in soup.find_all('a',{'class':"btn btn-lg btn-primary"}):
    restaurant = {}
    restaurant['name'] = row.text
    restaurant['slug'] = row.attrs['href'].split('/')[2]
    restaurants.append(restaurant)
df_restaurants = pd.DataFrame(restaurants)

In [88]:
# check the data
df_restaurants.head()

,name,slug
0,A&W Restaurants,aw-restaurants
1,Applebee's,applebees
2,Arby's,arbys
3,Atlanta Bread Company,atlanta-bread-company
4,Bojangle's Famous Chicken 'n Biscuits,bojangles-famous-chicken-n-biscuits


In [89]:
# check the size
df_restaurants.shape

(44, 2)

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

**Note**: Remove extra white space from each category

In [91]:
# create a food dictionary
foods = []
for restaurant in restaurants: # loop through all restaurants
    url_food = url + "restaurants/" + restaurant['slug'] + "/"
    res_food = requests.get(url_food) # construct url for each res
    print(f'Status code for {url_food}: {res_food.status_code}')
    soup_food = BeautifulSoup(res_food.content, 'lxml')
    for block in soup_food.find_all('tr')[1:]:
        food = {}
        food['name'] = block.find('td', {'itemprop': 'name'}).text.strip()
        food['calories'] = block.find('td', {'itemprop': 'calories'}).text.strip()
        food['category'] = block.find('a').text.strip()
        food['fat'] = block.find('td',{'itemprop': 'fatContent'}).text.strip()
        food['carbs'] = block.find('td',{'itemprop': 'carbohydrateContent'}).text.strip()
        food['restaurant'] = restaurant['name']
        foods.append(food)

Status code for http://chubbygrub.com/restaurants/aw-restaurants/: 200
Status code for http://chubbygrub.com/restaurants/applebees/: 200
Status code for http://chubbygrub.com/restaurants/arbys/: 200
Status code for http://chubbygrub.com/restaurants/atlanta-bread-company/: 200
Status code for http://chubbygrub.com/restaurants/bojangles-famous-chicken-n-biscuits/: 200
Status code for http://chubbygrub.com/restaurants/buffalo-wild-wings/: 200
Status code for http://chubbygrub.com/restaurants/burger-king/: 200
Status code for http://chubbygrub.com/restaurants/captain-ds/: 200
Status code for http://chubbygrub.com/restaurants/carls-jr/: 200
Status code for http://chubbygrub.com/restaurants/charleys-grilled-subs/: 200
Status code for http://chubbygrub.com/restaurants/chick-fil-a/: 200
Status code for http://chubbygrub.com/restaurants/chilis/: 200
Status code for http://chubbygrub.com/restaurants/chipotle-mexican-grill/: 200
Status code for http://chubbygrub.com/restaurants/churchs/: 200
Stat

### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [96]:
# create a dataframe
df_foods = pd.DataFrame(foods)

# re-order columns
df_foods = df_foods[['restaurant','category','name','calories','carbs','fat']]

In [97]:
# check the firs 5 rows
df_foods.head()

,restaurant,category,name,calories,carbs,fat
0,A&W Restaurants,Drinks,A&W® Diet Root Beer,0,0,0
1,A&W Restaurants,Drinks,A&W® Diet Root Beer,0,0,0
2,A&W Restaurants,Drinks,A&W® Diet Root Beer,0,0,0
3,A&W Restaurants,Drinks,A&W® Diet Root Beer,0,0,0
4,A&W Restaurants,Drinks,A&W® Diet Root Beer,0,0,0


In [98]:
# check the shape
df_foods.shape

(4977, 6)

### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [99]:
# Export to csv
df_foods.to_csv('chubbygrub_complete_food_list.csv', index=False)